# Multiple Dispatch

### Basic dispatch

In [2]:
f(a, b::Any) = "fallback"
f(a::Number, b::Number) = "a and b are both numbers"
f(a::Number, b) = "a is a number"
f(a, b::Number) = "b is a number"
f(a::Integer, b::Integer) = "a and b are both integers"

f (generic function with 5 methods)

In [3]:
methods(f)

# 5 methods for generic function "f":
f(a::Integer, b::Integer) in Main at In[2]:5
f(a::Number, b::Number) in Main at In[2]:2
f(a::Number, b) in Main at In[2]:3
f(a, b::Number) in Main at In[2]:4
f(a, b) in Main at In[2]:1

In [4]:
f(1.5, 2)

"a and b are both numbers"

In [5]:
f(1, "bar")

"a is a number"

In [6]:
f(1, 2)

"a and b are both integers"

In [7]:
f("foo", [1,2])

"fallback"

In [8]:
f(1, 2, 3)

LoadError: [91mMethodError: no method matching f(::Int64, ::Int64, ::Int64)[0m
Closest candidates are:
  f(::Integer, ::Integer) at In[2]:5
  f(::Number, ::Number) at In[2]:2
  f(::Number, ::Any) at In[2]:3
  ...[39m

### Ambiguities

In [9]:
g(a::Int, b::Number) = 1
g(a::Number, b::Int) = 2

g (generic function with 2 methods)

In [10]:
g(1, 2.5)

1

In [11]:
g(1.5, 2)

2

In [12]:
g(1, 2)

LoadError: [91mMethodError: g(::Int64, ::Int64) is ambiguous. Candidates:
  g(a::Number, b::Int64) in Main at In[9]:2
  g(a::Int64, b::Number) in Main at In[9]:1
Possible fix, define
  g(::Int64, ::Int64)[39m

### "Diagonal" dispatch

In [13]:
f(a::T, b::T) where {T<:Number} = "a and b are both $(T)s"

f (generic function with 6 methods)

In [14]:
methods(f)

# 6 methods for generic function "f":
f(a::Integer, b::Integer) in Main at In[2]:5
f(a::T, b::T) where T<:Number in Main at In[13]:1
f(a::Number, b::Number) in Main at In[2]:2
f(a::Number, b) in Main at In[2]:3
f(a, b::Number) in Main at In[2]:4
f(a, b) in Main at In[2]:1

In [15]:
f(big(1.5), big(2.5))

"a and b are both BigFloats"

In [16]:
f(big(1), big(2)) # <== integer rule is more specific

"a and b are both integers"

In [17]:
f(a::T, b::T) where {T<:Integer} = "both are $T integers"

f (generic function with 7 methods)

In [18]:
methods(f)

# 7 methods for generic function "f":
f(a::T, b::T) where T<:Integer in Main at In[17]:1
f(a::Integer, b::Integer) in Main at In[2]:5
f(a::T, b::T) where T<:Number in Main at In[13]:1
f(a::Number, b::Number) in Main at In[2]:2
f(a::Number, b) in Main at In[2]:3
f(a, b::Number) in Main at In[2]:4
f(a, b) in Main at In[2]:1

In [19]:
f(big(1), big(2))

"both are BigInt integers"

In [20]:
f("foo", "bar") # <== still doesn't apply to non-numbers

"fallback"

### Varargs methods

In [21]:
f(args::Number...) = "$(length(args))-ary heterogeneous call"
f(args::T...) where {T<:Number} = "$(length(args))-ary homogeneous call"
# f() = "0-ary homogeneous call"

f (generic function with 9 methods)

In [22]:
f(1)

"1-ary homogeneous call"

In [24]:
f(1, 2, 3)

"3-ary homogeneous call"

In [25]:
f(1, 1.5, 2)

"3-ary heterogeneous call"

In [26]:
f() # ???

"0-ary heterogeneous call"

In [27]:
f(1, 2) # <== previous 2-arg method is more specific

"both are Int64 integers"

In [28]:
f("foo") # <== doesn't apply to non-numbers

LoadError: [91mMethodError: no method matching f(::String)[0m
Closest candidates are:
  f(::Any, [91m::Number[39m) at In[2]:4
  f(::Any, [91m::Any[39m) at In[2]:1
  f([91m::T<:Integer[39m, [91m::T<:Integer[39m) where T<:Integer at In[17]:1
  ...[39m

### Method Ambiguities

In [ ]:
g(x::Number, y::Int) = 1
g(x::Int, y::Number) = 2

In [ ]:
g(1,2)

In [ ]:
g(x::Int, y::Int) =
  invoke(g, (Int,Number), x, y) + invoke(g, (Number,Int), x, y)

In [ ]:
g(1,2)

### Optional Arguments

In [29]:
h(x, y = 0) = 2x + 3y

h (generic function with 2 methods)

In [30]:
methods(h)

# 2 methods for generic function "h":
h(x) in Main at In[29]:1
h(x, y) in Main at In[29]:1

Shorthand for this:
```
h(x, y) = 2x + 3y
h(x) = h(x, 0)
```

### Keyword Arguments

In [31]:
k(x, y = 0; opt::Bool = false) = opt ? 2x+y : x+2y

k (generic function with 2 methods)

In [32]:
k(2)

2

In [33]:
k(2, 3)

8

In [34]:
k(2, opt=true)

4

In [35]:
k(2, 3, opt=true)

7

In [36]:
foo(x, y; mandatory::Bool=error("this is necessary!")) = mandatory ? 2x+y : x+2y

foo (generic function with 1 method)

In [40]:
foo(2,3)

LoadError: [91mthis is necessary![39m

In [38]:
methods(k)

# 2 methods for generic function "k":
k(x) in Main at In[31]:1
k(x, y; opt) in Main at In[31]:1

In [39]:
k(2, opt=true)

4

In [ ]:
m(a=0, b=1, c=2, d=3, e=4) = a+b+c+d+e

In [ ]:
m()

In [ ]:
methods(m)